In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch

model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-2.8B-v1.0")
# model = AutoModelForSeq2SeqLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.half()
model.to(device)
# tokenizer.to(device)


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(58944, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((256

In [7]:
prompt_template = 'You are a natural language understanding assistant tasked with creating sentences for a dataset. Your job is to read the given first sentence and then generate three different second sentences: one that is neutral, one that is a contradiction, and one that is an entailment. \
    You need to make sentences in korean. \
    \n This is examples: \
    sentence1	sentence2	gold_label \
    그리고 그가 말했다, "엄마, 저 왔어요."	그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.	neutral\n \
    그리고 그가 말했다, "엄마, 저 왔어요."	그는 한마디도 하지 않았다.	contradiction\n \
    그리고 그가 말했다, "엄마, 저 왔어요."	그는 엄마에게 집에 갔다고 말했다.	entailment\n \
    내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 잃었어요.	neutral\n \
    내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.	contradiction\n \
    내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	나는 내가 무엇을 할 것인지 확신할 수 없어서 내가 보고를 하도록 배정받은 워싱턴으로 갔다.	entailment\n \
    그는 갈 수 없었다.	그는 처음 초대를 받은 사람이고 그 경험을 즐겼다.	contradiction\n \
    그는 갈 수 없었다.	그는 참석하는 것을 허가받지 않았다.	entailment\n \
    그는 갈 수 없었다.	그는 박물관 개관식에 가는 것이 허용되지 않았다.	neutral\n \
    그리고 저는 "괜찮아" 하고, 그게 다였어요!	내가 승낙한 후에, 그것은 끝났다.	entailment\n \
    그리고 저는 "괜찮아" 하고, 그게 다였어요!	나는 아니라고 대답했고, 그것은 계속 이어졌다.	contradiction\n \
    그리고 저는 "괜찮아" 하고, 그게 다였어요!	내가 좋다고 말했을 때, 우리는 그날 결혼하기로 결정했다.	neutral\n \
    저는, 그냥 알아내려고 거기 있었어요.	나는 처음부터 그것을 잘 이해했다.	contradiction\n \
    저는, 그냥 알아내려고 거기 있었어요.	나는 돈이 어디로 갔는지 이해하려고 했어요.	neutral\n \
    저는, 그냥 알아내려고 거기 있었어요.	이해하려고 노력하고 있었어요.	entailmentsentence1 sentence2 golden\n \
    .\nHuman: {prompt}\nAssistant:\n'
text = '계획배합은 시험비빔을 해 정하고, 발주자 대리인의 지시에 따른다.'
model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors='pt').to(device)

outputs = model.generate(**model_inputs, max_new_tokens=256)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(output_text)

Setting `pad_token_id` to `eos_token_id`:58943 for open-end generation.


You are a natural language understanding assistant tasked with creating sentences for a dataset. Your job is to read the given first sentence and then generate three different second sentences: one that is neutral, one that is a contradiction, and one that is an entailment.     You need to make sentences in korean.     
 This is examples:     sentence1	sentence2	gold_label     그리고 그가 말했다, "엄마, 저 왔어요."	그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.	neutral
     그리고 그가 말했다, "엄마, 저 왔어요."	그는 한마디도 하지 않았다.	contradiction
     그리고 그가 말했다, "엄마, 저 왔어요."	그는 엄마에게 집에 갔다고 말했다.	entailment
     내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 잃었어요.	neutral
     내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.	contradiction
     내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.	나는 내가 무엇을 할 것인지 확신할 수 없어서 내가 보고를 하도록 배정받은 워싱턴으로 갔다.	entailment
     그는 갈 수 없었다.	그는 처음 초대를 받은 사람이고 그 경험을 즐겼다.	co